# 📊 Session 1: Pandas Fundamental

**Durasi:** 90 menit  
**Dataset:** RUP Paket Penyedia 2025

## 🎯 Tujuan Pembelajaran
Setelah sesi ini, Anda dapat:
1. Memuat dan melihat data dengan Pandas
2. Memilih dan memfilter data
3. Melakukan agregasi sederhana
4. Menangani missing values
5. Membuat summary statistics

## 1️⃣ Setup & Load Data

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# Konfigurasi display
pd.set_option('display.max_columns', 20)
pd.set_option('display.float_format', '{:.2f}'.format)

print(f"Pandas version: {pd.__version__}")

In [ ]:
# Load dataset
data_path = Path('../../../../datasets/rup/RUP-PaketPenyedia-Terumumkan-2025.parquet')
df = pd.read_parquet(data_path)

print(f"✅ Data loaded: {len(df):,} rows, {len(df.columns)} columns")

## 2️⃣ Eksplorasi Awal

In [ ]:
# Lihat 5 baris pertama
df.head()

In [ ]:
# Info struktur data
df.info()

In [ ]:
# Summary statistics untuk kolom numerik
df[['pagu']].describe()

## 3️⃣ Seleksi Data

In [ ]:
# Pilih kolom penting
key_cols = ['nama_paket', 'pagu', 'metode_pengadaan', 'jenis_pengadaan', 'nama_satker']
df_simple = df[key_cols].copy()
df_simple.head()

In [ ]:
# Filter: paket dengan pagu > 1 Miliar
high_value = df_simple[df_simple['pagu'] > 1_000_000_000]
print(f"Paket dengan pagu > 1M: {len(high_value):,}")
high_value.head()

In [ ]:
# Filter kombinasi: Tender dan pagu > 1M
tender_high = df_simple[
    (df_simple['pagu'] > 1_000_000_000) & 
    (df_simple['metode_pengadaan'] == 'Tender')
]
print(f"Paket Tender dengan pagu > 1M: {len(tender_high):,}")

## 4️⃣ Agregasi & GroupBy

In [ ]:
# Hitung total pagu per metode pengadaan
pagu_per_metode = df.groupby('metode_pengadaan')['pagu'].agg([
    ('jumlah_paket', 'count'),
    ('total_pagu_miliar', lambda x: x.sum() / 1e9),
    ('rata_rata_miliar', lambda x: x.mean() / 1e9)
]).round(2)

pagu_per_metode.sort_values('total_pagu_miliar', ascending=False)

In [ ]:
# Top 10 Satker berdasarkan total pagu
top_satker = df.groupby('nama_satker')['pagu'].agg([
    ('jumlah_paket', 'count'),
    ('total_pagu_miliar', lambda x: x.sum() / 1e9)
]).sort_values('total_pagu_miliar', ascending=False).head(10)

top_satker

## 5️⃣ Missing Values

In [ ]:
# Cek missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_summary = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct
}).query('missing_count > 0').sort_values('missing_count', ascending=False)

missing_summary.head(10)

## 6️⃣ Sorting & Ranking

In [ ]:
# 10 Paket dengan pagu terbesar
top_pagu = df.nlargest(10, 'pagu')[['nama_paket', 'pagu', 'metode_pengadaan', 'nama_satker']]
top_pagu['pagu_miliar'] = (top_pagu['pagu'] / 1e9).round(2)
top_pagu[['nama_paket', 'pagu_miliar', 'metode_pengadaan']]

## 📊 Summary Statistics

In [ ]:
# Statistik pagu
print("=" * 50)
print("STATISTIK PAGU PENGADAAN")
print("=" * 50)
print(f"Total Paket     : {len(df):,}")
print(f"Total Pagu      : Rp {df['pagu'].sum() / 1e12:.2f} Triliun")
print(f"Rata-rata Pagu  : Rp {df['pagu'].mean() / 1e6:.2f} Juta")
print(f"Median Pagu     : Rp {df['pagu'].median() / 1e6:.2f} Juta")
print(f"Pagu Terkecil   : Rp {df['pagu'].min():,.0f}")
print(f"Pagu Terbesar   : Rp {df['pagu'].max() / 1e9:.2f} Miliar")
print("=" * 50)

## 🎯 Latihan Mandiri

1. Hitung jumlah paket per jenis pengadaan
2. Temukan satker dengan jumlah paket terbanyak (bukan total pagu)
3. Filter paket dengan metode "E-Purchasing" dan pagu < 10 juta
4. Hitung persentase paket per metode pengadaan

In [ ]:
# Ruang untuk latihan